In [1]:
import sys
sys.path.insert(0,'/nfs/nfs2/users/riadoshi/bigvision_palivla/dlimp')
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

import tensorflow as tf
import numpy as np

from PIL import Image, ImageDraw
import mediapy
import re

from octo.data.oxe import make_oxe_dataset_kwargs
from octo.data.dataset import make_single_dataset

2024-12-25 19:12:19.802012: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-25 19:12:19.807243: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-25 19:12:19.819109: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735153939.838593 2506892 cuda_dnn.cc:8501] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735153939.844543 2506892 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-25 19:12:19.865443: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
# load chunk of dataset in based on ID
print("Loading dataset *******")
dataset = 'bridge_dataset'
dataset_kwargs = make_oxe_dataset_kwargs(dataset,"gs://rail-orca-central2/resize_256_256/")
dataset_kwargs["use_cot"]=True
dataset_kwargs["cot_data_path"]="gs://multi-robot-bucket2/data/generated_data"
dataset = make_single_dataset(
                                dataset_kwargs, 
                                frame_transform_kwargs=dict(
                                    resize_size={"primary": (256, 256)},
                                ),
                                train=True)
iterator = dataset.iterator()

Loading dataset *******
loading cot data
loaded cot data!
building the reasoning dict...
Example reasoning: 0_0 gripper <loc0264><loc0360>; silver tray <loc0788><loc0400><loc1020><loc0816><seg045><seg018><seg074><seg012><seg035><seg059><seg078><seg114><seg106><seg010><seg050><seg069><seg099><seg031><seg035><seg018>; gripper <loc0270><loc0360>; silver tray <loc0788><loc0404><loc1020><loc0824>; gripper <loc0277><loc0366>; silver tray <loc0788><loc0400><loc1020><loc0824>; gripper <loc0266><loc0376>; silver tray <loc0788><loc0404><loc1020><loc0824>; gripper <loc0326><loc0361>; silver tray <loc0792><loc0404><loc1020><loc0824>; 
Reasoning presence statistics [# has not, # has]: [14577, 38615]


2024-12-25 19:12:48.355030: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Cause: Unable to locate the source code of <function _gcd_import at 0x7f517f247400>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function _gcd_import at 0x7f517f247400>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function _gcd_import at 0x7f517f247400>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2024-12-25 19:12:56.900466: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


In [3]:
def plotted_gripper(traj):
    gripper_plans = []
    for reasoning in traj['reasonings']:
        # Find all occurrences of "gripper" followed by loc tokens
        reasoning = reasoning.decode('utf-8')
        matches = re.findall(r"gripper <loc(\d{4})><loc(\d{4})>", reasoning)
        gripper_plan = []
        
        for match in matches:
            # Extract the integers and normalize as specified
            y = float(match[0]) * 256 / 1024  # Normalize y
            x = float(match[1]) * 256 / 1024  # Normalize x
            gripper_plan.append([y, x])
        
        # Append the list of locations for this step
        gripper_plans.append(gripper_plan)

    return gripper_plans

def plot_gripper_plans(traj, step=5):
    gripper_plans = plotted_gripper(traj)
    observations = traj['observation']['image_primary'][step:step+5]
    grip_plan = gripper_plans[step:step+5][0]
    print(grip_plan)
    
    overlayed_frames = []
    for idx, (image, _) in enumerate(zip(observations, gripper_plans[step:step+5])):
        # Convert the image to a PIL format
        pil_image = Image.fromarray(image.squeeze())
        draw = ImageDraw.Draw(pil_image)
        
        # Overlay the gripper plan
        for y,x in grip_plan:
            # Draw a small circle for each gripper location
            draw.ellipse([(x - 4, y - 4), (x + 4, y + 4)], fill=(255, 0, 0))
        
        # Add the frame to the list
        overlayed_frames.append(np.array(pil_image))
    
    # Display the frames as a video using mediapy
    mediapy.show_video(overlayed_frames, fps=1)  # Slower FPS for clarity

In [18]:
def parse_object_bounding_boxes(reasoning):
    """
    Extract object bounding boxes from a reasoning string.
    """
    reasoning_str = reasoning.decode("utf-8")  # Decode byte string
    object_matches = re.findall(
        r"([a-zA-Z\s]+)<loc(\d{4})><loc(\d{4})><loc(\d{4})><loc(\d{4})>", reasoning_str
    )
    objects = {}
    for obj_name, y_min, x_min, y_max, x_max in object_matches:
        name = obj_name.strip()
        bbox = [
            [int(y_min) * 256 / 1024, int(x_min) * 256 / 1024],  # Top-left corner
            [int(y_max) * 256 / 1024, int(x_max) * 256 / 1024],  # Bottom-right corner
        ]
        if name not in objects:
            objects[name] = []
        objects[name].append(bbox)
    return objects

def plot_object_plans(traj, step=0):
    """
    Plot gripper and object plans over observations for the next 5 steps.
    """
    # Parse gripper and object plans
    object_bboxes = [
        parse_object_bounding_boxes(reasoning) for reasoning in traj['reasonings']
    ]
    
    # Extract the observations for the specified steps
    observations = traj['observation']['image_primary'][step:step+5]
    
    # Initialize a list to hold the frames
    overlayed_frames = []
    
    for idx, image in enumerate(observations):
        step_idx = step + idx
        pil_image = Image.fromarray(image.squeeze())
        draw = ImageDraw.Draw(pil_image)
        
        # Overlay object bounding boxes for the current step
        for obj_name, bboxes in object_bboxes[step].items():
            for (y_min, x_min), (y_max, x_max) in bboxes:
                draw.rectangle(
                    [(x_min, y_min), (x_max, y_max)],
                    outline=(0, 255, 0),
                    width=2
                )
                draw.text((x_min + 5, y_min + 5), obj_name, fill=(255, 255, 255))
        
        # Add the frame to the list
        overlayed_frames.append(np.array(pil_image))
    
    # Display the frames as a video using mediapy
    mediapy.show_video(overlayed_frames, fps=1)  # Slower FPS for clarity

In [35]:
traj = next(iterator)
print(traj['task']['language_instruction'][0])
plot_gripper_plans(traj, step=10)
plot_object_plans(traj, step=10)

b'move the silver pot to the lower right burner'
[[86.5, 190.0], [84.0, 200.0], [83.0, 201.0], [82.5, 205.0], [84.5, 211.0]]


This browser does not support the video tag.


This browser does not support the video tag.
